In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores import AstraDB
from langchain_astradb import AstraDBVectorStore, AstraDBSemanticCache

from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

from app import hf_embedding_model, vector_store, vec_retriever, llm, rag_chain

c:\Users\ASUS\miniconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.24it/s]


ঢাকায় ঘোরার জন্য অনেক কিছুই আছে!  আপনি পুরান ঢাকার ঐতিহাসিক স্থাপনা যেমন আহসান মঞ্জিল, লালবাগ কেল্লা, তারা মসজিদ, আরো অনেক কিছু দেখতে পারেন। আহসান মঞ্জিলের মার্বেলের মেঝে ও বারান্দা, বিভিন্ন কক্ষ,  সুন্দর বাগান দেখার মত। লালবাগ কেল্লার প্রাচীর, ফোঁকর, গোলাকার মঞ্চ  দেখার মত।  আপসাইড ডাউন গ্যালারীতেও যেতে পারেন, যদিও বর্তমানে সেটি বন্ধ।  শাহবাগে গিয়ে অপরাজেয় বাংলা দেখতে পারেন। পুরান ঢাকার বিখ্যাত খাবারও খেতে পারেন।  এছাড়াও আরো অনেক আকর্ষণীয় স্থান আছে ঢাকায়।  আপনার পছন্দ অনুযায়ী স্থান নির্বাচন করতে পারেন।  আমাদের ফেসবুক পেজে আরো তথ্য পাবেন।।


In [2]:
# credentials
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


os.environ['LANGSMITH_TRACING_V2']="true"
os.environ['LANGSMITH_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGSMITH_PROJECT']="travel_bot_bd"

llm_with_tools = llm.bind_tools([vec_retriever])
# https://youtu.be/y5GvqOi4bJQ?si=aZ41fbtc4_IGBqVJ

In [3]:
def rag_chain_llm(inputs:dict)->dict:
    msg = f"{inputs['question']}"
    res = rag_chain.invoke(msg)
    # print(res)
    return {"answer": res}

In [4]:
rag_chain_llm({"question":"পুরান ঢাকায় ঘোরার মত কি আছে?"})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.88it/s]


{'answer': 'পুরান ঢাকায় ঘোরার মতো অনেক কিছুই আছে!  আপনি যদি আহসান মঞ্জিল দেখতে যান, তাহলে কাছেই লালবাগ কেল্লা, তারা মসজিদ, আরও  অনেক ঐতিহাসিক স্থাপনা দেখতে পারবেন।  আর্মেনিয়ান চার্চ, হোসেনি দালান, বাহাদুর শাহ পার্ক (ভিক্টোরিয়া পার্ক) ও সদরঘাট লঞ্চঘাটও  ঘুরে দেখার মতো।  পুরান ঢাকার বিখ্যাত খাবারের দোকানগুলোও  চেখে দেখতে পারেন।  আহসান মঞ্জিলের সুন্দর গম্বুজ, মার্বেলের মেঝে,  খাবার ঘর, লাইব্রেরি,  দরবার হল  সবই দেখার মতো।  এছাড়াও, লালবাগ কেল্লার  ঐতিহাসিক গুরুত্ব ও স্থাপত্য  অসাধারণ।  ইদ্রাকপুর কেল্লা সম্পর্কেও লোকমুখে অনেক কথা প্রচলিত আছে।  শাহবাগে গেলে অপরাজেয় বাংলা দেখতে পারবেন।  আপনার ভ্রমণ আরও আনন্দময় হোক!।'}

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

eval_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [6]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langsmith import Client

client = Client()
dataset_name = "bd_travel_eval_dataset"
dataset = client.read_dataset(dataset_name=dataset_name)

# Provide the LLM to the evaluator
qa_evaluator = [LangChainStringEvaluator(evaluator="cot_qa", config={"llm": eval_llm})]


# Run the evaluation
experiment_results = evaluate(
    rag_chain_llm,
    dataset_name,
    qa_evaluator,
    client=client,
    experiment_prefix="cot_qa",
)


View the evaluation results for experiment: 'cot_qa-770b4149' at:
https://smith.langchain.com/o/3a43463f-5a0e-5086-bca2-6b8f06bc1910/datasets/1c1f035d-c3cb-44b4-a09b-d18e88cac258/compare?selectedSessions=176cc1bd-c188-4df8-8b1e-8502ebcf26be




Batches: 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]
7it [00:38,  5.53s/it]


In [7]:
qa_evaluator = [LangChainStringEvaluator(evaluator="context_qa", config={"llm": eval_llm})]


# Run the evaluation
experiment_results = evaluate(
    rag_chain_llm,
    dataset_name,
    qa_evaluator,
    client=client,
    experiment_prefix="context_qa",
)

View the evaluation results for experiment: 'context_qa-821e9927' at:
https://smith.langchain.com/o/3a43463f-5a0e-5086-bca2-6b8f06bc1910/datasets/1c1f035d-c3cb-44b4-a09b-d18e88cac258/compare?selectedSessions=419aff07-81dd-45e0-9bcf-05f94ba93f07




Batches: 100%|██████████| 1/1 [00:00<00:00,  3.60it/s]
7it [00:29,  4.23s/it]


In [ ]:
qa_evaluator = [LangChainStringEvaluator(evaluator="score_string", config={"criteria": "relevance", 
                                                                           "normalize_by": 10, 
                                                                           "llm": eval_llm})]


# Run the evaluation
experiment_results = evaluate(
    rag_chain_llm,
    dataset_name,
    qa_evaluator,
    client=client,
    experiment_prefix="score_string",
)

In [10]:
qa_evaluator = [LangChainStringEvaluator(evaluator="criteria", config={"criteria":"conciseness",
    "llm": eval_llm})]

# Run the evaluation
experiment_results = evaluate(
    rag_chain_llm,
    dataset_name,
    qa_evaluator,
    client=client,
    experiment_prefix="conciseness",
)

View the evaluation results for experiment: 'conciseness-6c7130ba' at:
https://smith.langchain.com/o/3a43463f-5a0e-5086-bca2-6b8f06bc1910/datasets/1c1f035d-c3cb-44b4-a09b-d18e88cac258/compare?selectedSessions=a90e024b-cafb-42b0-939b-cecc2ead2b9f




Batches: 100%|██████████| 1/1 [00:00<00:00,  6.68it/s]
7it [00:42,  6.09s/it]
